In [1]:
from scripts.utils.yahtzee import Game
from scripts.utils.constants import CATEGORIES
from scripts.strategies.greedy_strategy import GreedyStrategy
from scripts.strategies.rules_based_strategy import RulesBasedStrategy
from scripts.strategies.random_strategy import RandomStrategy
from itertools import combinations_with_replacement, combinations, product
from collections import Counter
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.ticker as mtick
import math
import os
import pyarrow
from scipy.stats import t

import matplotlib.pyplot as plt

## Background: How do the potential scores of various rolls differ?

In [ ]:
gs = GreedyStrategy()
all_rolls, all_rolls_scores = gs._score_all_rolls()

In [ ]:
# Create histograms of potential score for each category
n_rows = 5
n_cols = 3
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 20))
axes = axes.flatten()
for i, c in zip(range(0, 13), CATEGORIES):
    bins = np.linspace(0, 50)

    vals = all_rolls_scores[:,i]

    mu = vals.mean()
    sigma = vals.std()
    pct_zero = len(vals[vals==0])/len(vals) * 100
    textstr = '\n'.join((
        r'$\mathrm{E(Score)}=%.2f$' % (mu, ),
        r'$\mathrm{Percent\ zero}=%.1f\%%$' % (pct_zero, ),
        r'$\sigma=%.2f$' % (sigma, )))

    # these are matplotlib.patch.Patch properties
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

    # place a text box in upper left in axes coords
    axes[i].text(0.6, 0.95, textstr, transform=axes[i].transAxes, fontsize=10,
    verticalalignment='top', bbox=props)
    
    axes[i].hist(vals, bins=bins, alpha = 0.6, linewidth=5, density=True)
    axes[i].set_title(f'{c}')
    axes[i].set_xlabel('Score')
    axes[i].set_ylabel('Pr(Score)')
    axes[i].yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

# Remove extra graphs
for i in range(len(CATEGORIES), len(axes)):
    fig.delaxes(axes[i])


plt.suptitle('Distribution of Scores by Category for a Single Roll', y=1, fontsize=18)
plt.tight_layout()

plt.savefig('figures/single_roll_category_scores.png')
plt.close()


In [ ]:
iterations = 100000

# 1. Simple random strategy

## a. Purely random

In [ ]:
game_states_purely_random = []
random_strategy = RandomStrategy(tiered_strategy=False)

# Use different seed for each iteration
for i in tqdm(range(1, iterations * 5, 5), miniters=iterations/1000):
    game_states_purely_random.append(random_strategy.run_strategy(start_seed=i, rng_seed = int(math.pow(2, 32) - 1 - i), tiered_strategy=False))

## b. Tiered strategy

In [ ]:
game_states_tiered_random = []
random_strategy_tiered = RandomStrategy(tiered_strategy=True)

# Use different seed for each iteration
for i in tqdm(range(1, iterations * 5, 5), miniters=iterations/1000):
    game_states_tiered_random.append(random_strategy_tiered.run_strategy(start_seed=i, rng_seed = int(math.pow(2, 32) - 1 - i), tiered_strategy=True))

# 2. Simple greedy strategy
Prioritize expected score of next most imminent roll

## a. Greedy strategy

In [ ]:
# game_states_greedy = []
# greedy_strategy = GreedyStrategy()
# # Use different seed for each iteration
# for i in tqdm(range(1, iterations * 5, 5), miniters=iterations/1000):
#     game_states_greedy.append(greedy_strategy.run_strategy(start_seed=i, prioritize_upper_section=True))

# 3. Rules-based approach

## a. Based solely on Glenn

In [ ]:
game_states_rules_glenn = []
rules_based_strategy_glenn = RulesBasedStrategy()

# Use different seed for each iteration
for i in tqdm(range(1, iterations * 5, 5), miniters=iterations/1000):
    game_states_rules_glenn.append(rules_based_strategy_glenn.run_strategy(i))

## b. Prioritize upper section bonus

In [ ]:
game_states_rules_upper = []
rules_based_strategy_upper = RulesBasedStrategy()
# Use different seed for each iteration
for i in tqdm(range(1, iterations * 5, 5), miniters=iterations/1000):
    game_states_rules_upper.append(rules_based_strategy_upper.run_strategy(start_seed=i, prioritize_upper_section=True))

## c. Prioritize Yahtzee

In [ ]:
game_states_rules_yahtzee = []
rules_based_strategy_yahtzee = RulesBasedStrategy()

# Use different seed for each iteration
for i in tqdm(range(1, iterations * 5, 5), miniters=iterations/1000):
    game_states_rules_yahtzee.append(rules_based_strategy_yahtzee.run_strategy(start_seed=i, prioritize_yahtzee=True))

# 4. Analyze results for final score

## a. Save results and convert into dataframes

In [ ]:
# Convert to dataframes and save locally (for further analysis)
dfs = {}
for out, label in zip([
                        game_states_purely_random, 
                        game_states_tiered_random, 
                        game_states_greedy, 
                        game_states_rules_glenn,
                        game_states_rules_upper,
                        game_states_rules_yahtzee,
                        ],
                [
                'purely_random', 
                'tiered_random', 
                'greedy', 
                'rules_glenn',
                'rules_upper',
                'rules_yahtzee'
                ]):
    df = pd.DataFrame(out)
    df.drop(columns=['available_categories', 'potential_scores', 'rolls_remaining', 'dice_values'], inplace=True)

    df.to_csv(f'data/{label}_runs.csv.gzip', compression='gzip', sep=";")

    dfs[label] = df.copy(deep=True)

In [ ]:
## Sample code for loading locally saved rresultss
# import os
# os.makedirs('data', exist_ok=True)

# filename = 'data/greedy_no_upper_runs.csv.gzip'

# df = pd.read_csv(filename, compression='gzip', sep=";")
# df['final_score'].mean()

## b. Within-strategy distribution of final score and CI

In [ ]:
# Get CI around mean for final score
def get_mean_ci(values: pd.Series, alpha: float=0.05):
    """
    Returns mean, sample standard deviation, and two-sided CI for given alpha
    """
    n = len(values)
    sample_mean = np.mean(values)

    # DDOF = 1 returns sample standard deviation (vs. population standard deviation)
    sample_std = np.std(values, ddof = 1)

    # Get t-critical value
    t_crit = t.ppf(1-alpha/2, n-1)

    ci_lower = float(sample_mean - t_crit * sample_std / math.sqrt(n))
    ci_upper = float(sample_mean + t_crit * sample_std / math.sqrt(n))

    return sample_mean, sample_std, ci_lower, ci_upper


# Create histograms of potential score for each category
def output_histograms(dfs: dict, column: str = 'final_score', column_name: str = 'Final Score'):
    n_rows = 5
    n_cols = 3
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 20))
    axes = axes.flatten()
    max_val = np.max([v[column].max() for v in dfs.values()])
    for i, c in zip(range(len(dfs)), dfs.keys()):
        bins = np.linspace(0, np.ceil(max_val))

        vals = dfs[c][column]

        mu = vals.mean()
        sigma = vals.std(ddof=1)
        textstr = '\n'.join((
            r'$\mathrm{Sample\ Mean}=%.2f$' % (mu, ),
            r'$\mathrm{Sample\ Std\ Dev}=%.2f$' % (sigma, )))

        # these are matplotlib.patch.Patch properties
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

        # place a text box in upper left in axes coords
        axes[i].text(0.5, 0.95, textstr, transform=axes[i].transAxes, fontsize=10,
        verticalalignment='top', bbox=props)
            
        
        axes[i].hist(vals, bins=bins, alpha = 0.5, linewidth=5, density=True)
        axes[i].set_title(f'{c}')
        axes[i].set_xlabel(column_name)
        axes[i].set_ylabel(f'Pr({column_name})')
        axes[i].yaxis.set_major_formatter(mtick.PercentFormatter(100.0))

    # Remove extra graphs
    for i in range(len(dfs), len(axes)):
        fig.delaxes(axes[i])

    return fig, axes


In [ ]:
# Final score
variants = [
                'purely_random', 
                'tiered_random', 
                'greedy', 
                'rules_glenn',
                'rules_upper',
                'rules_yahtzee'
                ]

final_score_output = pd.DataFrame(columns = ['strategy', 'sample_mean', 'sample_std', 'ci_95p_lower', 'ci_95p_upper'])
for v in variants:
    sample_mean, sample_std, ci_lower, ci_upper = get_mean_ci(dfs[v]['final_score'])
    final_score_output.loc[len(final_score_output.index), ] = [v, sample_mean, sample_std, ci_lower, ci_upper]

final_score_output.to_clipboard(index=False)

# Get histogram
fig, axes = output_histograms(dfs, 'final_score', 'Final Score')
plt.suptitle('Distribution of Final Score by Strategy', y=1, fontsize=18)
plt.tight_layout()
plt.savefig('figures/final_score_by_category.png')
plt.close()


In [ ]:
# CIs for Yahtzees
yahtzee_output = pd.DataFrame(columns = ['strategy', 'sample_mean', 'sample_std', 'ci_95p_lower', 'ci_95p_upper'])
for v in variants:
    sample_mean, sample_std, ci_lower, ci_upper = get_mean_ci(dfs[v]['num_yahtzees'])
    yahtzee_output.loc[len(yahtzee_output.index), ] = [v, sample_mean, sample_std, ci_lower, ci_upper]

yahtzee_output.to_clipboard(index=False)

# Get histogram
fig, axes = output_histograms(dfs, 'num_yahtzees', 'Number fo Yahtzees')
plt.suptitle('Distribution of Number of Yahtzees by Strategy', y=1, fontsize=18)
plt.tight_layout()
plt.savefig('figures/num_yahtzees_by_category.png')
plt.close()


In [ ]:
# CIs for earning final bonus
upper_output = pd.DataFrame(columns = ['strategy', 'sample_mean', 'sample_std', 'ci_95p_lower', 'ci_95p_upper'])
for v in variants:
    dfs[v]['has_earned_upper_bonus'] = (dfs[v]['upper_points_remaining'] == 0).astype(int)
    sample_mean, sample_std, ci_lower, ci_upper = get_mean_ci(dfs[v]['has_earned_upper_bonus'])
    upper_output.loc[len(upper_output.index), ] = [v, sample_mean, sample_std, ci_lower, ci_upper]

upper_output.to_clipboard(index=False)

# Get histogram
fig, axes = output_histograms(dfs, 'has_earned_upper_bonus', 'Has Earned Upper Bonus')
plt.suptitle('Distribution of % Earning Upper Bonus by Strategy', y=1, fontsize=18)
plt.tight_layout()
plt.savefig('figures/upper_bonus_by_category.png')
plt.close()

## c. Compare among strategies

In [ ]:
# Implement Kim & Nelson 2001 to find best m set of strategies
def kim_nelson_sequential(procedures: dict, delta: float, alpha: float=0.05, n0: int=10, m: int = 1, quiet=True):
    """
    Implements Kim & Nelson (2001)
    
    Inputs:
        procedures: Dictionary of procedure name: values
        delta: Indifference zone
        alpha: Confidence interval
        n0: Original number of observations
        m: number of systems desired

    Returns:
        List of best procedures
        Number of observations needed to obtain best procedures
    """
    I = procedures.copy()
    k = len(procedures.keys())

    ## 1. Calculate constants for c = 1 (recommended)
    c = 1
    eta = 1/2 * (((2 * alpha)/(k - 1))**(-2/(n0-1)) - 1)
    h_squared = 2 * c * eta * (n0 - 1)

    ## 2. Calculate maximum sample size Ni
    # Get sample mean for n0 observations
    sample_means = {i: np.mean(v.iloc[:n0]) for i, v in I.items()}

    # Generate all potential combinations of systems
    pairs = list(combinations(I.keys(),2))
    pairs = [p for p in pairs if p[0] != p[1]]

    # Calculate sample variances for each pair
    S_squared_il = {}
    for i, l in pairs:
        sample_var = 0.0
        for o in range(n0):
            sample_var += (I[i].iloc[o] - I[l].iloc[o] - (sample_means[i] - sample_means[l]))**2
        S_squared_il[(i, l)] = sample_var / (n0 - 1)

    # Calculate Ni (max sample size)
    Ni = {k: np.floor(v * h_squared / (delta**2)) for k, v in S_squared_il.items()}
    Ni_max = max(Ni.values())

    # If n0 > max Ni, select system with highest sample mean
    if n0 > Ni_max:
        max_mean = max(sample_means.values())
        return [k for k, v in sample_means.items() if v == max_mean], n0

    r = n0
    ## 3. Screening
    while True:
        I_old = I.copy()
        sample_means = {i: np.mean(v.iloc[:r]) for i, v in I.items()}

        pairs = list(combinations(I_old.keys(), 2))
        pairs = [p for p in pairs if p[0] != p[1]]

        # Calculate W_il: how far the sample mean from system i can drop below the sample means of the other systems without being eliminated
        W_il = {}
        for p in pairs:
            S_squared_il_p = S_squared_il[p]
            W_il[p] = max(0, delta/(2 * c * r) * (h_squared * S_squared_il_p / delta**2 - r))

        # Screen for surviving systems
        surviving_systems = []
        for i in I_old.keys():
            keep = True
            for p in pairs:
                if not((p[0] == i) or (p[1] == i)) or not(keep):
                    continue
                l = p[0] if p[0] != i else p[1]

                Xi = sample_means[i]
                Xl = sample_means[l]

                if Xi < (Xl - W_il[p]):
                    if not quiet:
                        print(f'{i} dropped: worse than {l}')
                    keep = False
                    continue
            
            if keep:
                surviving_systems.append(i)

        I = {k: v for k, v in I_old.items() if k in surviving_systems}
        ## 4. Stopping rule
        # If length of I is less than or equal to m, return remaining systems
        if len(I.keys()) <= m:
            return list(I.keys()), r

        r += 1
        if not quiet:
            print(f'r = {r}, systems remaining = {surviving_systems}')
        # If r = max(Ni) + 1, return  system with largest sample mean
        if r >= (Ni_max + 1):
            max_mean = max([m for k, m in sample_means.items() if k in surviving_systems])
            return [k for k, v in sample_means.items() if v == max_mean and k in surviving_systems], r

In [ ]:
# Implement Kim & Nelson
procedures_final_score = {v: dfs[v]['final_score'] for v in variants}
procedures_upper_section = {v: dfs[v]['has_earned_upper_bonus'] for v in variants}
procedures_yahtzee = {v: dfs[v]['num_yahtzees'] for v in variants}

output_df = pd.DataFrame(columns=['variable', 'delta', 'm', 'winners', 'reps'])
print("Testing final score")
# Try various values of delta
for d in [0.1, 0.2, 0.5, 1, 2, 3, 4, 5, 6]:
    w2, n2 = kim_nelson_sequential(procedures=procedures_final_score, delta = d, quiet=True, m=2)
    w1, n1 = kim_nelson_sequential(procedures=procedures_final_score, delta = d, quiet=True, m=1)
    output_df.loc[len(output_df.index), ] = ['final_score', d, 2, w2, n2]
    output_df.loc[len(output_df.index), ] = ['final_score', d, 1, w1, n1]

print()
print("Testing upper section bonus")
# Try various values of delta
for d in [0.01, 0.05, 0.1, 0.2]:
    w2, n2 = kim_nelson_sequential(procedures=procedures_upper_section, delta = d, quiet=True, m=2)
    w1, n1 = kim_nelson_sequential(procedures=procedures_upper_section, delta = d, quiet=True, m=1)
    output_df.loc[len(output_df.index), ] = ['has_earned_upper_bonus', d, 2, w2, n2]
    output_df.loc[len(output_df.index), ] = ['has_earned_upper_bonus', d, 1, w1, n1]

print()
print("Testing # Yahtzees")
# Try various values of delta
for d in [0.01, 0.05, 0.1, 0.2]:
    w2, n2 = kim_nelson_sequential(procedures=procedures_yahtzee, delta = d, quiet=True, m=2)
    w1, n1 = kim_nelson_sequential(procedures=procedures_yahtzee, delta = d, quiet=True, m=1)
    output_df.loc[len(output_df.index), ] = ['num_yahtzees', d, 2, w2, n2]
    output_df.loc[len(output_df.index), ] = ['num_yahtzees', d, 1, w1, n1]

output_df.to_clipboard(index=False)

# 5. Further analysis

## a. Distribution of score by category and order in which categories are scored

In [ ]:
# Focus only on top two strategies (Greedy & Glenn rules-based)
from collections import defaultdict

def get_category_scores_and_order(df):
    category_scores_df = pd.DataFrame(df['scores'].to_list(), columns = CATEGORIES)
    score_order = dfs['greedy']['score_order'].to_list()

    order_round_to_category_dict = defaultdict(list)
    order_category_to_round_dict = defaultdict(list)

    # score order by col
    for i, r in df.iterrows():
        for j, v in enumerate(r['score_order']):
            # Standardize Yahtzee bonuses
            if 'yahtzee, ' in v:
                order_round_to_category_dict[j+1].append(v.replace('yahtzee, ', ''))
                order_category_to_round_dict[v.replace('yahtzee, ', '')].append(j+1)
            else:
                order_round_to_category_dict[j+1].append(v)
                order_category_to_round_dict[v].append(j+1)

    order_round_to_category_df = pd.DataFrame(order_category_to_round_dict)
    return category_scores_df, order_round_to_category_df

variants = [
                'purely_random', 
                'tiered_random', 
                'greedy', 
                'rules_glenn',
                'rules_upper',
                'rules_yahtzee'
                ]

dfs_categories = {}
for v in variants:
    dfs_categories[v] = {}
    category_scores_df, order_round_to_category_df = get_category_scores_and_order(dfs[v])
    dfs_categories[v]['category_scores'] = category_scores_df
    dfs_categories[v]['category_rounds'] = order_round_to_category_df

In [ ]:
def histograms_score(df: pd.DataFrame, column_name: str = 'Category Score'):
    """
    Generates histograms for each potential category
    """
    n_rows = 5
    n_cols = 3
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 20))
    axes = axes.flatten()

    cols = df.columns

    max_val = max([df[c].max() for c in cols])
    for i, c in zip(range(len(cols)), CATEGORIES):
        bins = np.linspace(0, np.ceil(max_val.astype(float)))

        vals = df[c]

        mu = vals.mean()
        sigma = vals.std(ddof=1)
        if column_name == 'Category Score':
            pct_zero = (vals == 0).astype(int).sum()/len(vals)
            textstr = '\n'.join((
                r'$\mathrm{Sample\ Mean}=%.2f$' % (mu, ),
                r'$\mathrm{Pct\ Zero}=%.2f$' % (pct_zero, ),
                r'$\mathrm{Sample\ Std\ Dev}=%.2f$' % (sigma, )))

        else:
            textstr = '\n'.join((
                r'$\mathrm{Sample\ Mean}=%.2f$' % (mu, ),
                r'$\mathrm{Sample\ Std\ Dev}=%.2f$' % (sigma, )))

        # these are matplotlib.patch.Patch properties
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

        # place a text box in upper left in axes coords
        axes[i].text(0.5, 0.95, textstr, transform=axes[i].transAxes, fontsize=10,
        verticalalignment='top', bbox=props)
            
        axes[i].hist(vals, bins=bins, alpha = 0.5, linewidth=5, density=True)
        axes[i].set_title(f'{c}')
        axes[i].set_xlabel(column_name)
        axes[i].set_ylabel(f'Pr({column_name})')
        axes[i].yaxis.set_major_formatter(mtick.PercentFormatter(100.0))

    # Remove extra graphs
    for i in range(len(cols), len(axes)):
        fig.delaxes(axes[i])

    return fig, axes


In [ ]:
# Outputs histograms for score by category and order in which categories are scored
# CIs for score by category, # of zeros, and order in which categories are scored
category_score_output_df = pd.DataFrame(columns = ['strategy', 'category', 'sample_mean', 'sample_std', 'ci_95p_lower', 'ci_95p_upper'])
category_zeros_output_df = pd.DataFrame(columns = ['strategy', 'category', 'sample_mean', 'sample_std', 'ci_95p_lower', 'ci_95p_upper'])
category_order_output_df = pd.DataFrame(columns = ['strategy', 'category', 'sample_mean', 'sample_std', 'ci_95p_lower', 'ci_95p_upper'])

for k, d in dfs_categories.items():
    df_categories = d['category_scores']
    df_rounds = d['category_rounds']

    fig, axes = histograms_score(df_categories, 'Category Score')
    plt.suptitle(f'Distribution of Score by Category - {k}', y=1, fontsize=18)
    plt.tight_layout()
    plt.savefig(f'figures/score_by_category_{k}.png')
    plt.close()

    fig, axes = histograms_score(df_rounds, 'Round Earned')
    plt.suptitle(f'Distribution of Round Earned by Category - {k}', y=1, fontsize=18)
    plt.tight_layout()
    plt.savefig(f'figures/score_by_category_{k}.png')
    plt.close()

    # CIs
    for c in CATEGORIES:
        vals_scores = df_categories[c]
        vals_zeros = (vals_scores == 0).astype(int)
        vals_rounds = df_rounds[c]

        sample_mean_s, sample_std_s, ci_lower_s, ci_upper_s = get_mean_ci(vals_scores)
        sample_mean_z, sample_std_z, ci_lower_z, ci_upper_z = get_mean_ci(vals_zeros)
        sample_mean_r, sample_std_r, ci_lower_r, ci_upper_r = get_mean_ci(vals_rounds)

        category_score_output_df.loc[len(category_score_output_df.index), ] = [k, c, sample_mean_s, sample_std_s, ci_lower_s, ci_upper_s]
        category_zeros_output_df.loc[len(category_zeros_output_df.index), ] = [k, c, sample_mean_z, sample_std_z, ci_lower_z, ci_upper_z]
        category_order_output_df.loc[len(category_order_output_df.index), ] = [k, c, sample_mean_r, sample_std_r, ci_lower_r, ci_upper_r]


In [ ]:
category_score_output_df.to_clipboard(index=False)

In [ ]:
category_zeros_output_df.to_clipboard(index=False)

In [ ]:
category_order_output_df.to_clipboard(index=False)

## b. When should chance be scored?